Imports

In [2]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset
from os import chdir, remove, close, listdir, path
from pydub import AudioSegment 


/home/rodoggx/iResearch-2024/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# https://archive.liveatc.net/kjfk/KJFK-Twr-Jun-24-2024-0000Z.mp3

$baseUrl = "https://archive.liveatc.net/kjfk/KJFK-Twr-Jun-24-2024-"

# Initialize time as a string
$time = "0000"

$client = New-Object System.Net.WebClient

# Loop until time is less than 24 hours
while ($time -ne "2330") {
	$url = $baseUrl + $time + "Z.mp3"

	Write-Output "Downloading $url"

	try {
		# Download the file
		$client.DownloadFile($url, "/home/rodoggx/iResearch-2024/ATC-recordings/KJFK-Twr-Jun-24-2024-" + $time + "Z.mp3")
	} catch {
		Write-Output "File not found"
	}
    # Convert string to DateTime object for easy manipulation
    $timeObject = [DateTime]::ParseExact($time, "HHmm", $null)

    # Add 30 minutes
    $timeObject = $timeObject.AddMinutes(30)

    # Convert DateTime object back to string
    $time = $timeObject.ToString("HHmm")
}

In [6]:
chdir("/home/rodoggx/iResearch-2024/ATC-recordings")
for foo in listdir():
    input_file = foo
    output_file = foo.rsplit('.', 1)[0] + '.wav'

    # convert mp3 file to wav file 
    sound = AudioSegment.from_mp3(input_file) 
    sound.export(output_file, format="wav")
    
    remove(foo)

In [4]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    torch_dtype=torch_dtype,
    device=device,
)

dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]

chdir("/home/rodoggx/iResearch-2024/ATC-recordings")
for i in range(0, 4):
    foo = listdir()[i]
    result = pipe(foo)
    print(foo)
    print(result)
    print("\n----------------------------------------------------------\n")

    with open("output.txt", "a") as f:
        f.write(str(foo))
        f.write(str(result))
        f.write("\n----------------------------------------------------------\n")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/rodoggx/iResearch-2024/venv/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:480: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


KJFK-Twr-Jun-24-2024-0830Z.mp3.wav
{'text': " you you JetBlue 1823, new, departure 125.7 25.7, good day, JetBlue 1823 And Ram, good morning. JetBlue 1114 is with you coming up on Bravo, making the turnaround for gate 29. That was a beautiful message, but it's just stuck in me, Tom. Oh, sorry about that. you you you you Well, there's a piece of left bubble in the chair. you you you you you you you you you you you you you you you ... ... Tower, Chibblew 782 is just outside Salpo for 22L. Chibblew 782, Canadian Tower, 1250, I'm 90, runway 22L, settle in. Atlanta to left, it was 7-8-2. you you you you you Roger, 7-82, Hotel, across to your right, what's the gate number? IAT ramp. We have to enter Gulf, I should have said that, Gulf, IAT ramp, JetBlue, 7-82. Roger, JetBlue, 7-82, right on Bravo with me. Alright, Hotel, across to your right, right on Bravo with you, JetBlue, 7-82. you 2348, I left, 22 left, morning. 2348, 28,000, morning, 124, GRI, 9-0, 122 left, clear to land. Clear to land

/home/rodoggx/iResearch-2024/venv/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:480: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


KJFK-Twr-Jun-24-2024-2100Z.mp3.wav
{'text': " JetBlue 784. JetBlue 784, at Alpha 4, join Alpha to the ramp and monitor ground. All right, at Alpha 4, we'll join Alpha to the ramp, monitor ground, JetBlue 784. Kennedy Tower, helicopter 3, Bravo Lima, Governor's Island 800, Park conduit GAT. Helicopter 3, Bravo Lima, Kennedy Tower, Sparks 0331, cleared as requested, EW1000, speak. 0331, cleared as requested, EW1000, 3, Bravo Lima. you ... ... ... ... ... ... ... ... ... ... ... ... ... ... Yep, we're short 58.02, monitor ground point 9. Monitor ground point 9, Texas, stand by for the day. Thank you. Out of the blue is 5-7-0, 7-7-0-11 Tower, for 7-7-7, just a quick update, I'll descend as soon as I get just on the west side of these fields, right in front of me Say again please? I'll be descending in like one mile, 7-0-11 I said 11 Bravo heavy exit left taxi left on Bravo this frequency Left on Bravo remain your frequency at 11 Bravo Alpha 4, left on Alpha, 11 Bravo. Hey Bravo Lima landin

/home/rodoggx/iResearch-2024/venv/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:480: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


KJFK-Twr-Jun-24-2024-1830Z.mp3.wav
{'text': " Alright Skyline, Delta 419 and Bravo Kennedy Tower, wind 300 at 22, GUS 34, runway 31R, set a line 31R, set a line Delta 419, Bravo Zip Blue 356, turn left Victor, taxi to the ramp, straight ahead to uniform, and then a left turn on Alpha, monitor ground, 29 Victor, left turn on uniform and alpha, and oh, congrats, that was great. That was great, thanks. Tower, good afternoon, American, 332 on the ILS, runway 31R. NASA, 332, Kennedy Tower, good afternoon, 4213, 6 miles and trail, the heavy air bus 350, wind 300 at 22, gust 34, runway 31R, cleared to land. 31R, cleared to land, American, 332. you This is 1770, turn left, Whiskey, Whiskey, taxi to the ramp, left turn, Bravo, taxiway Alpha 4, taxiway Alpha, monitor ground 0.9. Whiskey, Whiskey, Bravo, Whiskey, Alpha 4, over to our ramp, Joplin 70. Yeah, taxiway Alpha 4, join Alpha. Alpha 4, join Alpha, sorry about that, Joplin 70. Delta 419 Bravo, say airspeed. Delta 419 Bravo, you're signed t

ValueError: Soundfile is either not in the correct format or is malformed. Ensure that the soundfile has a valid audio file extension (e.g. wav, flac or mp3) and is not corrupted. If reading from a remote URL, ensure that the URL is the full address to **download** the audio file.